In [13]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from helper_functions_pipe_testing import *
from sklearn.metrics import  f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import *
from helper_functions_KP import *

#Instructions for the pipeline Requires two inputs for training: - Mass spec data with corresponding NP surface characteristics and experimental conditions (time, concentration) - NetsurfP and Biopython data that has been precalculated - X characteristics to predict
pipeline Take mass spec spreadsheet Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration Merge with Proteome data to get file that has Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration,Mass,Length,Sequence Calculate protein features using biopython Merge with NSP data to get all protein features
Split into X and Y dataset with Entries as labels

In [4]:
# Pull together Proteomic data
in_dir="Input_data/Proteomic data/abundance/"
all_file='Input_data/Proteomic data/abundance/Intensity _all20230202.xlsx'
#combine Mass Spec data input into one excel spreadsheet - Entry - Abundance labeled by NP Unique ID
#Abundance as a percent
#take files in_dir and combine then into one pandas df (raw_MS_data)
# files = os.listdir(in_dir)
# for i,f in enumerate(files):
#     if i==0:
#         raw_MS_data=pd.read_excel(in_dir+f,header=0)
#     else:
#         temp = pd.read_excel(in_dir+f,header=0)
#         raw_MS_data=raw_MS_data.merge(temp,how='outer',on='Entry')
raw_MS_data=pd.read_excel(all_file,header=0)
# melt the df to make it an accession number, NPUNID, Abundance dataset
raw_MS_data = pd.melt(raw_MS_data, id_vars=['Entry'],var_name='Sample_num', value_name='Abundance')
#remove prots that were added due to merge
raw_MS_data=raw_MS_data.dropna()
###Bring in controls (MS data for serums)##
controls=pd.read_excel('Input_data/Proteomic data/controls_combined.xlsx',header=0)
MS_data_controls = pd.merge(raw_MS_data,controls,how='left', on='Entry')
###Bring in Uniprot_data,NSPdata and NP data##
uniprot_filepath='Input_data/BioPython_data/Combined_biopyCalcs.xlsx'
uniprot_dat=pd.read_excel(uniprot_filepath,header=0)
NSPfilePath='Input_data/NetSurfP_data/Combined.xlsx'
NSP_data=pd.read_excel(NSPfilePath)
###Bring in NP data and merge to get complete NP dataset###
NP_filepath='Input_data/NPs/NP_Database.xlsx'
NPUNdata=pd.read_excel(NP_filepath,header=0,sheet_name='NPUNID')
NPprop=pd.read_excel(NP_filepath,header=0,sheet_name='NP_Props')
NPdata=pd.merge(NPUNdata,NPprop,how="left",on='NPID')
NPdata.dropna(inplace=True)
#calculate Enrichment
#####MAYBE add binning here to to keep negative results and improve capapbilities######
MS_data_controls['Enrichment']= np.log2(MS_data_controls['Abundance']/MS_data_controls['Abundance_Controls'])
MS_data=MS_data_controls.drop(columns=['Abundance','Abundance_Controls'])
raw_prop_data=pd.merge(MS_data, uniprot_dat.drop_duplicates(subset=['Entry']), how='left',on='Entry')

Protein_data_complete = pd.merge(raw_prop_data, NSP_data.drop_duplicates(subset=['Entry']),how='left', on='Entry') #merges netsurfp features and biopython features
Protein_data_complete.fillna(0,inplace=True)
#creates new column called asa_sum_normalized which is the asa_sum value divide by the mass of the protein
for df in [Protein_data_complete]:
    for col in ['asa_sum']:
        df[col+'_normalized'] = df[col] / df['Mass']

data_complete= pd.merge(Protein_data_complete,NPdata,how='inner', on='Sample_num')
data_complete.drop(columns=['notes','Notes','NPUNID'],inplace=True)
data_complete.fillna(0,inplace=True)
data_complete= data_complete.replace([-np.inf],'-12')
data_complete=data_complete.replace([np.inf],'12')
data_complete2=pd.get_dummies(data_complete, columns=['Core Material','Surface_Ligand'])


#set labels (what we are trying to predict) as Enrichment column
labels=data_complete2['Enrichment'].copy()
#make it one dimenisional
labels=np.ravel(labels)
#drop qualitative, not neccessary, and label columns
df=data_complete2.drop(['Entry','Sequence','NPID','Enrichment','Ligands','Protein Source','Sample_num','Unnamed: 5','Raw_FileID'],axis=1)
#create df without bonus NSP columns (remove total_exposed)
df_NSP_drop=df.copy()
to_drop=df_NSP_drop.filter(like='total_exposed_')
df_NSP_drop.drop(columns=to_drop,inplace=True)

C:\Users\kmp95\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [5]:
#create normalized dataframe
df_norm=df.copy()
for col in df_norm.columns:
    if col == 'asa_sum_normalized':
        continue
    df_norm[col]=df_norm[col]/df_norm[col].max()
    print(col)

Length
Mass
frac_aa_A
frac_aa_C
frac_aa_D
frac_aa_E
frac_aa_F
frac_aa_G
frac_aa_H
frac_aa_I
frac_aa_K
frac_aa_L
frac_aa_M
frac_aa_N
frac_aa_P
frac_aa_Q
frac_aa_R
frac_aa_S
frac_aa_T
frac_aa_V
frac_aa_W
frac_aa_Y
molecular_weight
aromaticity
instability_index
flexibility_mean
flexibility_std
flexibility_var
flexibility_max
flexibility_min
flexibility_median
isoelectric_point
secondary_structure_fraction_helix
secondary_structure_fraction_turn
secondary_structure_fraction_sheet
secondary_structure_fraction_disordered
gravy
fraction_exposed
fraction_buried
fraction_exposed_nonpolar_total
fraction_exposed_nonpolar_exposed
fraction_exposed_polar_total
fraction_exposed_polar_exposed
rsa_mean
rsa_median
rsa_std
asa_sum
fraction_total_exposed_A
fraction_total_exposed_C
fraction_total_exposed_D
fraction_total_exposed_E
fraction_total_exposed_F
fraction_total_exposed_G
fraction_total_exposed_H
fraction_total_exposed_I
fraction_total_exposed_K
fraction_total_exposed_L
fraction_total_exposed_M
fra

In [11]:
df_norm.fillna(0,inplace=True)

In [12]:
df_list=[df,df_norm,df_NSP_drop]
out_name=['df','norm','nspdrop']
df_listnew=[]
x=0
for i in df_list:
    df_listnew.append(feat_elim_rand(i,labels,out_name[x],20,250,3))
    x+=1

In [25]:
estimators=np.arange(5,1000,5)
out_name=np.arange(5,1000,5)
print(out_name)
out_list=[0]*200
for i in range(len(estimators)):
    out_list[i]=rand_forest_reg_fit(df_listnew[2],labels,out_name[i],0.20,estimators[i])

[  5  10  15  20  25  30  35  40  45  50  55  60  65  70  75  80  85  90
  95 100 105 110 115 120 125 130 135 140 145 150 155 160 165 170 175 180
 185 190 195 200 205 210 215 220 225 230 235 240 245 250 255 260 265 270
 275 280 285 290 295 300 305 310 315 320 325 330 335 340 345 350 355 360
 365 370 375 380 385 390 395 400 405 410 415 420 425 430 435 440 445 450
 455 460 465 470 475 480 485 490 495 500 505 510 515 520 525 530 535 540
 545 550 555 560 565 570 575 580 585 590 595 600 605 610 615 620 625 630
 635 640 645 650 655 660 665 670 675 680 685 690 695 700 705 710 715 720
 725 730 735 740 745 750 755 760 765 770 775 780 785 790 795 800 805 810
 815 820 825 830 835 840 845 850 855 860 865 870 875 880 885 890 895 900
 905 910 915 920 925 930 935 940 945 950 955 960 965 970 975 980 985 990
 995]


In [37]:
# print(out_list[2])
a=pd.DataFrame(list(zip(estimators,out_list)), columns=['number of estimators','accuracy'])
a.to_excel("Output_data/estimatorandaccuracy_20feats.xlsx")

In [ ]:
#plot distrubition of (feat) with -labels and dsitrubition with positive values
# frac_aa_E,K,R, flexibility_min, Isoelectric Point, gravy, fraction_exposed_L,M,R,S,T,Y, zeta potential
#plot PCA of features and labels after RFE
#plot loss of features after RFE